<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #6: Train machine learning models</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: March 9, 2022</h5>

# Set up and requirements

### Import Python packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Set the directory of the libraries
import sys, os
from pathlib import Path
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from ipyfilechooser import FileChooser

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.server_utils as s_utils
import kso_utils.project_utils as p_utils
import kso_utils.t6_utils as t6
import kso_utils.t12_utils as t12
from src.prepare_zooniverse import frame_aggregation
from kso_utils.zooniverse_utils import retrieve_zoo_info, populate_subjects, populate_agg_annotations

# Model-specific imports
import train
import test

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

ModuleNotFoundError: No module named 'kso_utils.project_utils'

### Choose a project

In [7]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'), value='Koster_Sea…

In [8]:
project = p_utils.find_project(project_name=project_name.value)

### Initiate SQL database and populate sites, movies and species

In [86]:
# Initiate db
db_info_dict = t_utils.initiate_db(project)

Enter your username for SNIC server········
Enter your password for SNIC server········
Updated sites
Updated movies
Updated species


In [87]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project)

Enter your Zooniverse user········
Enter your Zooniverse password········


### Retrieve Zooniverse information

In [88]:
#if not "zoo_info_dict" in vars():
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project = project, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects", "workflows", "classifications"])

Retrieving subjects from Zooniverse
subjects were retrieved successfully
Retrieving workflows from Zooniverse
workflows were retrieved successfully
Retrieving classifications from Zooniverse
classifications were retrieved successfully
Updated subjects
The database has a total of 4295 frame subjects and 7362 clip subjects have been updated


### Specify Zooniverse parameters for frame aggregation

In [27]:
# Display a selectable list of workflow names and a list of versions of the workflow of interest
workflows_df = zoo_info_dict["workflows"]
wm = t12.WidgetMaker(workflows_df)
wm

WidgetMaker(children=(IntText(value=0, description='Number of workflows:', style=DescriptionStyle(description_…

Output()

In [89]:
# Retrieve classifications from the workflow of interest
class_df = t12.get_classifications(wm.checks,
                                     workflows_df, 
                                     'frame', 
                                     zoo_info_dict["classifications"], 
                                     db_info_dict["db_path"],
                                     project)

      subject_ids    id https_location
0        39431409  <NA>            NaN
1        39431413  <NA>            NaN
2        39431414  <NA>            NaN
3        39431419  <NA>            NaN
4        39431420  <NA>            NaN
...           ...   ...            ...
4594     59645870  <NA>            NaN
4595     59646443  <NA>            NaN
4596     59645310  <NA>            NaN
4597     59645375  <NA>            NaN
4598     59645352  <NA>            NaN

[1764 rows x 3 columns]
There are 1764 classifications out of 15022 missing subject info. Maybe the subjects have been removed from Zooniverse?
Zooniverse classifications have been retrieved


In [90]:
# Specify the agreement threshold required among cit scientists
agg_params = t12.choose_agg_parameters("frame")

FloatSlider(value=0.8, continuous_update=False, description='Aggregation threshold:', max=1.0, readout_format=…

IntSlider(value=3, continuous_update=False, description='Min numbers of users:', max=15, min=1, style=SliderSt…

FloatSlider(value=0.8, continuous_update=False, description='Object threshold:', max=1.0, readout_format='.1f'…

FloatSlider(value=0.5, continuous_update=False, description='IOU Epsilon:', max=1.0, readout_format='.1f', sty…

FloatSlider(value=0.8, continuous_update=False, description='Inter user agreement:', max=1.0, readout_format='…

In [91]:
agg_class_df, raw_class_df = t12.aggregrate_classifications(
                                    class_df, 'frame', project, agg_params)

Aggregrating the classifications
496 classifications aggregated out of 2027 unique subjects available


In [92]:
# Add annotations to db
populate_agg_annotations(agg_class_df, 'frame', project)

Updated agg_annotations_frame


### Step 0: Specify important paths and training parameters

In [55]:
# Specify output path where processed data will be stored
fc = t_utils.choose_folder(".", "output")

FileChooser(path='.', filename='', title='HTML(value='Choose location of output')', show_hidden='False', use_d…

In [56]:
# Store selected output path
output_folder = fc.selected

In [57]:
# Choose species of interest for model training
class_list = t6.choose_classes(db_info_dict["db_path"])

SelectMultiple(description='Species', index=(0,), options=('Angular crab', 'Ascidians (any species)', 'Bivalve…

In [96]:
# Store selected classes of interest
cl = list(class_list.value)

In [62]:
# Determine your training parameters
percentage_test, batch_size, epochs, conf_thres = t6.choose_test_prop()

### Step 1: Prepare the aggregated data

In [97]:
# Run the preparation script
frame_aggregation(project, db_info_dict, output_folder, percentage_test.value, cl, (720, 540))

  0%|          | 0/237 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/pims/pyav_reader.py:232: MethodDeprecationWarning: VideoStream.seek is deprecated.
  self._stream.seek(timestamp + self._first_pts)
  5%|▍         | 11/237 [00:12<04:18,  1.14s/it]


IndexError: index out of range

### Step 2: Train the model with selected parameters

In [116]:
# Fix important paths
data_path = [str(Path(output_folder, _)) for _ in os.listdir(output_folder) if _.endswith(".yaml") and "hyp" not in _][-1]
hyps_path = str(Path(output_folder, "hyp.yaml"))
weights = "/usr/src/app/data_dir/weights/yolov5m.pt"

# Choose folder that will contain the different model runs
project_path = FileChooser('/cephyr/NOBACKUP/groups/snic2021-6-9/models/koster-ml')

# Project-specific information
entity = "koster"
exp_name = "sgu_bm_model"
display(project_path)

FileChooser(path='/cephyr/NOBACKUP/groups/snic2021-6-9/models/koster-ml', filename='', title='HTML(value='', l…

In [12]:
# Train model with given configuration

In [126]:
train.run(entity=entity, data=data_path, hyp=hyps_path, weights=weights, 
          project=project_path.selected, name=exp_name,
          img_size=[720, 540], batch=int(batch_size.value),
          epochs=epochs.value, workers=4, single_cls=True, cache_images=True)

train: weights=/usr/src/app/data_dir/weights/yolov5m.pt, cfg=, data=/usr/src/app/data_dir/data_output/SGU_14:19:23.yaml, hyp=/usr/src/app/data_dir/data_output/hyp.yaml, epochs=100, batch_size=16, img_size=[720, 540], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket=, cache_images=True, image_weights=False, device=, multi_scale=False, single_cls=True, adam=False, sync_bn=False, workers=4, project=/cephyr/NOBACKUP/groups/snic2021-6-9/models/sgu/, entity=koster, name=sgu_bm_model, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, batch=16
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
requirements: /usr/src/app/data_dir/latest-yolo-test/notebooks/requirements.txt not found, check failed.


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jannesg (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


train: Scanning '/usr/src/app/data_dir/data_output/train' images and labels...272 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 272/272 [00:00<00:00, 3853.48it/s]
train: Caching images (0.3GB): 100%|██████████| 272/272 [00:00<00:00, 605.42it/s]
val: Scanning '/usr/src/app/data_dir/data_output/valid' images and labels...68 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 68/68 [00:00<00:00, 1269.42it/s]
val: Caching images (0.0GB): 100%|██████████| 68/68 [00:00<00:00, 415.58it/s]


Plotting labels... 


  0%|          | 0/17 [00:00<?, ?it/s]

...

                 all         68        373      0.498      0.571      0.525      0.324


     99/99     8.29G    0.0191   0.03205         0   0.05115        88       736: 100%|██████████| 17/17 [00:09<00:00,  1.72it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 3/3 [00:03<00:00,  1.07s/it]


                 all         68        373      0.524      0.549      0.526      0.323
Optimizer stripped from /cephyr/NOBACKUP/groups/snic2021-6-9/models/sgu/sgu_bm_model105/weights/last.pt, 42.4MB
Optimizer stripped from /cephyr/NOBACKUP/groups/snic2021-6-9/models/sgu/sgu_bm_model105/weights/best.pt, 42.4MB


train/box_loss,0.0191
train/obj_loss,0.03205
train/cls_loss,0.0
metrics/precision,0.52416
metrics/recall,0.5493
metrics/mAP_0.5,0.52552
metrics/mAP_0.5:0.95,0.32291
val/box_loss,0.03206
val/obj_loss,0.12562
val/cls_loss,0.0
x/lr0,0.00101


train/box_loss,█▇▆▆▅▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train/obj_loss,▆▇███▇▆▆▆▅▅▅▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁
train/cls_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/precision,▁▁▂▂▄▆▆▄▄▆▆▇▆▆▇▇▇▆▆▇█▇█▇▆▇█▇█▇█▇▇█▇▇▆▇▇▇
metrics/recall,▃▁▁▂▄▅▅▅▆▇█▇▇▇▇▇▇▇▇▇▇█▇▇▇▆▆▇▆▇▇▇▇▆▇▇█▇█▇
metrics/mAP_0.5,▁▁▁▂▃▅▆▅▅▆▇█▇▆█▇▇▇▇▇▇███▆▇███▇█▇▇▇▇█▇▇█▇
metrics/mAP_0.5:0.95,▁▁▁▁▂▄▄▃▃▅▇▇▅▆▇▆▆▆▆▇▇▇▇█▅▇██████▇███████
val/box_loss,█▇▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/obj_loss,▂▄▅▅▅▃▂▂▂▂▂▁▁▂▂▂▃▃▃▃▃▅▄▄█▆▄▅▅▅▆▆▇▆▇▆▇███
val/cls_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
x/lr0,▁▂▂▃▃▄▅▅▆▆▆▇▇▇█████████▇▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂


### Step 3: Evaluate model performance on test set

In [127]:
# Choose model
eval_model = FileChooser(project_path.selected)
display(eval_model)

FileChooser(path='/cephyr/NOBACKUP/groups/snic2021-6-9/models/sgu', filename='', title='HTML(value='', layout=…

In [128]:
# Find trained model weights
tuned_weights = f"{Path(project_path.selected, eval_model.selected, 'weights', 'best.pt')}"

In [ ]:
# Evaluate YOLOv3 Model on Unseen Test data for mAP metric

In [117]:
test.run(data=data_path, weights=tuned_weights, conf_thres=conf_thres.value, imgsz=640)

val: Scanning '/usr/src/app/data_dir/data_output/valid.cache' images and labels... 56 found, 12 missing, 0 empty, 0 corrupted: 100%|██████████| 68/68 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning '/usr/src/app/data_dir/data_output/valid.cache' images and labels... 56 found, 12 missing, 0 empty, 0 corrupted: 100%|██████████| 68/68 [00:00<?, ?it/s]
val: Scanning '/usr/src/app/data_dir/data_output/valid.cache' images and labels... 56 found, 12 missing, 0 empty, 0 corrupted: 100%|██████████| 68/68 [00:01<?, ?it/s]
               Class     I

                 all         68        192      0.902       0.24       0.23      0.124
Speed: 0.2ms pre-process, 9.3ms inference, 1.0ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs/test/exp


((0.9019607843137255,
  0.23958333333333334,
  0.23044666654540605,
  0.12408858931047799,
  0.0,
  0.0,
  0.0),
 array([    0.12409]),
 (0.21427168565637925, 9.343031574698054, 0.9574504459605497))

### Transfer model to web app server (for API use)

In [129]:
import getpass

In [130]:
server_user = getpass.getpass('Enter your server user')
server_pass = getpass.getpass('Enter your server password')

Enter your server user········
Enter your server password········


In [131]:
t6.transfer_model("bm_nmi18_enhanced", eval_model.selected, "koster/sgu", server_user, server_pass)

In [ ]:
# (Experimental) : Enhance annotations using trained model

In [121]:
import detect
detect.run(weights=tuned_weights, source=output_folder+"/images", imgsz=640, conf_thres=0.02, save_txt=True)

image 1/340 /usr/src/app/data_dir/data_output/images/nmi18_11002 0 Lat 6228074.31 Lon 631688.14 Pan -150.0 Tilt 0.0 Hdg 21.0 18_07_26 16_29_490208_patch_3_4.jpg: 480x640 4 blue_musselss, Done. (0.028s)
image 2/340 /usr/src/app/data_dir/data_output/images/nmi18_11002 0 Lat 6228074.31 Lon 631688.14 Pan -150.0 Tilt 0.0 Hdg 21.0 18_07_26 16_29_490208_patch_4_3.jpg: 480x640 5 blue_musselss, Done. (0.026s)
image 3/340 /usr/src/app/data_dir/data_output/images/nmi18_11002 18 Lat 6228074.29 Lon 631687.55 Pan -110.0 Tilt -50.0 Hdg 62.0 18_07_26 16_32_200226_patch_2_3.jpg: 480x640 2 blue_musselss, Done. (0.023s)
image 4/340 /usr/src/app/data_dir/data_output/images/nmi18_11002 18 Lat 6228074.29 Lon 631687.55 Pan -110.0 Tilt -50.0 Hdg 62.0 18_07_26 16_32_200226_patch_3_2.jpg: 480x640 8 blue_musselss, Done. (0.023s)
image 5/340 /usr/src/app/data_dir/data_output/images/nmi18_11002 18 Lat 6228074.29 Lon 631687.55 Pan -110.0 Tilt -50.0 Hdg 62.0 18_07_26 16_32_200226_patch_3_3.jpg: 480x640 8 blue_mussel

image 45/340 /usr/src/app/data_dir/data_output/images/nmi18_11004 6 Lat 6234628.73 Lon 639901.45 Pan 70.1 Tilt -27.0 Hdg 166.1 18_07_29 13_39_350008_patch_2_2.jpg: 480x640 8 blue_musselss, Done. (0.012s)
image 46/340 /usr/src/app/data_dir/data_output/images/nmi18_11004 6 Lat 6234628.73 Lon 639901.45 Pan 70.1 Tilt -27.0 Hdg 166.1 18_07_29 13_39_350008_patch_2_3.jpg: 480x640 4 blue_musselss, Done. (0.012s)
image 47/340 /usr/src/app/data_dir/data_output/images/nmi18_11004 6 Lat 6234628.73 Lon 639901.45 Pan 70.1 Tilt -27.0 Hdg 166.1 18_07_29 13_39_350008_patch_2_4.jpg: 480x640 6 blue_musselss, Done. (0.012s)
image 48/340 /usr/src/app/data_dir/data_output/images/nmi18_11004 6 Lat 6234628.73 Lon 639901.45 Pan 70.1 Tilt -27.0 Hdg 166.1 18_07_29 13_39_350008_patch_3_1.jpg: 480x640 9 blue_musselss, Done. (0.012s)
image 49/340 /usr/src/app/data_dir/data_output/images/nmi18_11004 6 Lat 6234628.73 Lon 639901.45 Pan 70.1 Tilt -27.0 Hdg 166.1 18_07_29 13_39_350008_patch_3_2.jpg: 480x640 10 blue_muss

image 87/340 /usr/src/app/data_dir/data_output/images/nmi18_16002 3 Lat 6222248.46 Lon 633935.09 Pan -59.9 Tilt -27.1 Hdg 187.1 18_05_30 11_26_450028_patch_4_2.jpg: 480x640 9 blue_musselss, Done. (0.012s)
image 88/340 /usr/src/app/data_dir/data_output/images/nmi18_16002 6 Lat 6222248.88 Lon 633935.08 Pan 70.1 Tilt -27.1 Hdg 322.1 18_05_30 11_27_200031_patch_4_0.jpg: 480x640 4 blue_musselss, Done. (0.012s)
image 89/340 /usr/src/app/data_dir/data_output/images/nmi18_16002 6 Lat 6222248.88 Lon 633935.08 Pan 70.1 Tilt -27.1 Hdg 322.1 18_05_30 11_27_200031_patch_4_1.jpg: 480x640 3 blue_musselss, Done. (0.012s)
image 90/340 /usr/src/app/data_dir/data_output/images/nmi18_16002 6 Lat 6222248.88 Lon 633935.08 Pan 70.1 Tilt -27.1 Hdg 322.1 18_05_30 11_27_200031_patch_4_2.jpg: 480x640 9 blue_musselss, Done. (0.012s)
image 91/340 /usr/src/app/data_dir/data_output/images/nmi18_16002 6 Lat 6222248.88 Lon 633935.08 Pan 70.1 Tilt -27.1 Hdg 322.1 18_05_30 11_27_200031_patch_4_4.jpg: 480x640 5 blue_muss

image 129/340 /usr/src/app/data_dir/data_output/images/nmi18_16018 3 Lat 6230742.29 Lon 641534.64 Pan -60.6 Tilt -27.1 Hdg 112.4 18_07_29 12_40_120028_patch_4_2.jpg: 480x640 6 blue_musselss, Done. (0.012s)
image 130/340 /usr/src/app/data_dir/data_output/images/nmi18_17006 0 Lat 6233400.74 Lon 647873.77 Pan -150.0 Tilt 0.0 Hdg 22.0 18_05_16 16_08_230049_patch_4_3.jpg: 480x640 3 blue_musselss, Done. (0.012s)
image 131/340 /usr/src/app/data_dir/data_output/images/nmi18_17006 18 Lat 6233400.73 Lon 647873.65 Pan -110.0 Tilt -50.0 Hdg 63.0 18_05_16 16_11_080067_patch_4_0.jpg: 480x640 4 blue_musselss, Done. (0.012s)
image 132/340 /usr/src/app/data_dir/data_output/images/nmi18_17006 6 Lat 6233400.73 Lon 647873.41 Pan 70.1 Tilt -27.1 Hdg 244.1 18_05_16 16_09_260055_patch_4_2.jpg: 480x640 2 blue_musselss, Done. (0.012s)
image 133/340 /usr/src/app/data_dir/data_output/images/nmi18_17006 6 Lat 6233400.73 Lon 647873.41 Pan 70.1 Tilt -27.1 Hdg 244.1 18_05_16 16_09_260055_patch_4_3.jpg: 480x640 6 blu

image 172/340 /usr/src/app/data_dir/data_output/images/nmi18_17019 3 Lat 6238050.12 Lon 653455.16 Pan -60.1 Tilt -27.1 Hdg 100.9 18_05_20 12_09_510155_patch_4_1.jpg: 480x640 5 blue_musselss, Done. (0.012s)
image 173/340 /usr/src/app/data_dir/data_output/images/nmi18_17019 3 Lat 6238050.12 Lon 653455.16 Pan -60.1 Tilt -27.1 Hdg 100.9 18_05_20 12_09_510155_patch_4_2.jpg: 480x640 9 blue_musselss, Done. (0.012s)
image 174/340 /usr/src/app/data_dir/data_output/images/nmi18_17019 6 Lat 6238050.13 Lon 653455.39 Pan 69.7 Tilt -27.1 Hdg 228.7 18_05_20 12_10_260158_patch_4_1.jpg: 480x640 13 blue_musselss, Done. (0.012s)
image 175/340 /usr/src/app/data_dir/data_output/images/nmi18_17019 6 Lat 6238050.13 Lon 653455.39 Pan 69.7 Tilt -27.1 Hdg 228.7 18_05_20 12_10_260158_patch_4_2.jpg: 480x640 3 blue_musselss, Done. (0.012s)
image 176/340 /usr/src/app/data_dir/data_output/images/nmi18_17019 6 Lat 6238050.13 Lon 653455.39 Pan 69.7 Tilt -27.1 Hdg 228.7 18_05_20 12_10_260158_patch_4_3.jpg: 480x640 10 b

image 214/340 /usr/src/app/data_dir/data_output/images/nmi18_19001 18 Lat 6233371.39 Lon 647983.34 Pan -110.0 Tilt -50.0 Hdg 194.0 18_05_20 16_38_490044_patch_4_1.jpg: 480x640 7 blue_musselss, Done. (0.013s)
image 215/340 /usr/src/app/data_dir/data_output/images/nmi18_19001 18 Lat 6233371.39 Lon 647983.34 Pan -110.0 Tilt -50.0 Hdg 194.0 18_05_20 16_38_490044_patch_4_3.jpg: 480x640 7 blue_musselss, Done. (0.012s)
image 216/340 /usr/src/app/data_dir/data_output/images/nmi18_19001 3 Lat 6233371.38 Lon 647983.10 Pan -59.9 Tilt -27.1 Hdg 240.1 18_05_20 16_36_260029_patch_4_0.jpg: 480x640 7 blue_musselss, Done. (0.012s)
image 217/340 /usr/src/app/data_dir/data_output/images/nmi18_19001 3 Lat 6233371.38 Lon 647983.10 Pan -59.9 Tilt -27.1 Hdg 240.1 18_05_20 16_36_260029_patch_4_1.jpg: 480x640 6 blue_musselss, Done. (0.012s)
image 218/340 /usr/src/app/data_dir/data_output/images/nmi18_19001 3 Lat 6233371.38 Lon 647983.10 Pan -59.9 Tilt -27.1 Hdg 240.1 18_05_20 16_36_260029_patch_4_2.jpg: 480x64

image 257/340 /usr/src/app/data_dir/data_output/images/nmi18_20007 6 Lat 6235843.76 Lon 654589.27 Pan 70.1 Tilt -27.1 Hdg 200.1 18_07_26 09_44_150150_patch_4_4.jpg: 480x640 1 blue_mussels, Done. (0.012s)
image 258/340 /usr/src/app/data_dir/data_output/images/nmi18_20015 0 Lat 6232939.89 Lon 650773.62 Pan -129.9 Tilt 0.0 Hdg 89.1 18_06_16 16_04_090051_patch_4_0.jpg: 480x640 8 blue_musselss, Done. (0.013s)
image 259/340 /usr/src/app/data_dir/data_output/images/nmi18_20015 0 Lat 6232939.89 Lon 650773.62 Pan -129.9 Tilt 0.0 Hdg 89.1 18_06_16 16_04_090051_patch_4_1.jpg: 480x640 5 blue_musselss, Done. (0.012s)
image 260/340 /usr/src/app/data_dir/data_output/images/nmi18_20015 0 Lat 6232939.89 Lon 650773.62 Pan -129.9 Tilt 0.0 Hdg 89.1 18_06_16 16_04_090051_patch_4_2.jpg: 480x640 10 blue_musselss, Done. (0.013s)
image 261/340 /usr/src/app/data_dir/data_output/images/nmi18_20015 0 Lat 6232939.89 Lon 650773.62 Pan -129.9 Tilt 0.0 Hdg 89.1 18_06_16 16_04_090051_patch_4_3.jpg: 480x640 7 blue_muss

image 298/340 /usr/src/app/data_dir/data_output/images/nmi18_23010 6 Lat 6234368.51 Lon 656371.57 Pan 70.1 Tilt -27.1 Hdg 136.1 18_06_02 13_09_330187_patch_4_1.jpg: 480x640 9 blue_musselss, Done. (0.012s)
image 299/340 /usr/src/app/data_dir/data_output/images/nmi18_23010 6 Lat 6234368.51 Lon 656371.57 Pan 70.1 Tilt -27.1 Hdg 136.1 18_06_02 13_09_330187_patch_4_2.jpg: 480x640 9 blue_musselss, Done. (0.012s)
image 300/340 /usr/src/app/data_dir/data_output/images/nmi18_23010 9 Lat 6234368.53 Lon 656372.16 Pan 157.1 Tilt -49.5 Hdg 227.1 18_06_02 13_10_000190_patch_4_2.jpg: 480x640 5 blue_musselss, Done. (0.012s)
image 301/340 /usr/src/app/data_dir/data_output/images/nmi18_23010 9 Lat 6234368.53 Lon 656372.16 Pan 157.1 Tilt -49.5 Hdg 227.1 18_06_02 13_10_000190_patch_4_3.jpg: 480x640 4 blue_musselss, Done. (0.012s)
image 302/340 /usr/src/app/data_dir/data_output/images/nmi18_24031 0 Lat 6221165.92 Lon 644651.71 Pan -129.9 Tilt 0.0 Hdg 230.1 18_06_13 11_19_420092_patch_4_2.jpg: 480x640 15 bl

image 340/340 /usr/src/app/data_dir/data_output/images/nmi18_37034 9 Lat 6226847.88 Lon 663893.58 Pan 157.1 Tilt -50.0 Hdg 30.1 18_06_17 11_24_340011_patch_4_2.jpg: 480x640 11 blue_musselss, Done. (0.013s)
Results saved to runs/detect/exp
340 labels saved to runs/detect/exp/labels
Done. (10.241s)


In [123]:
# Choose runs
runs = FileChooser(".")
display(runs)

FileChooser(path='.', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='False',…

In [125]:
!mv {output_folder}"/labels" {output_folder}"/labels_org"
!mv {runs.selected}"/labels" {output_folder}"/labels"

In [ ]:
#END